In [19]:
import torch
import numpy as np
import pandas as pd
# Importer la fonction de splitting des données de scikit learn
from sklearn.model_selection import train_test_split
# Importer Dataset de torch.utils.data
from torch.utils.data import Dataset
# Importer DataLoader de torch.utils.data
from torch.utils.data import DataLoader
# Importer le module nn
import torch.nn as nn
# Définir une fonction d'optimisation des coût: Adam par exemple. On devra définir un learning rate. On choisira 0.001.
import torch.optim as optim

In [20]:
# Créer une classe qui hérite de Dataset et redéfinit les méthodes 
class Covid_Dataset(Dataset):
    def __init__(self, data, targets):
        super(Covid_Dataset, self)
        self.data = data
        self.targets = targets
    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]
    def __len__(self):
        return len(self.targets)

In [ ]:
df = pd.read_excel("Data-FakeRealCOVID.xlsx")
newDF = pd.read_csv("newDF.csv")

In [ ]:
df.label.hist(xlabelsize=20,figsize=[5,5])

In [ ]:
newDF = newDF.drop('Unnamed: 0',axis=1)

In [ ]:
newDF

In [ ]:
def redDim(dfVec,seuil1,seuil2):
    subDf = dfVec[dfVec != 0]
    map = subDf.count(axis = 0) <= seuil1 
    map2 = subDf.count(axis = 0) >= (dfVec.shape[0] - seuil2) 
    resultDf = dfVec.drop(dfVec.columns[map+map2],axis=1)
    columns_dropped=dfVec.columns[map+map2]
    
    return resultDf

In [ ]:
labels = []
for label in df.label.to_numpy():
  if label == 'real':
    labels.append(1)
  else:
    labels.append(0)
targets=torch.tensor(labels)
targets

In [ ]:
targets.dtype

In [ ]:
data=torch.tensor(newDF.to_numpy())


In [ ]:
data

In [ ]:
# Importer la fonction de splitting des données de scikit learn
from sklearn.model_selection import train_test_split


In [ ]:
# Créer 4 tensors en résultat du splitting: données d'apprentissage, données de test, labels d'apprentissage, et labels de test  
atrain_data, test_data, atrain_targets, test_targets = train_test_split(data, targets, test_size=0.2)
atrain_data.shape

In [ ]:
# Créer 4 tensors en résultat du splitting: données d'apprentissage, données de validation, labels d'apprentissage, et labels de validation  
train_data, validation_data, train_targets, validation_targets = train_test_split(atrain_data, atrain_targets, test_size=1000)
train_data.shape

In [ ]:

# Afficher la taille de chaque tensor
print(train_data.shape, train_targets.shape)
print(validation_data.shape, validation_targets.shape)
print(test_data.shape,test_targets.shape)
print(train_data.shape[1])

In [ ]:
# Importer Dataset de torch.utils.data
from torch.utils.data import Dataset

In [ ]:
# Créer une classe qui hérite de Dataset et redéfinit les méthodes 
class Covid_Dataset(Dataset):
    def __init__(self, data, targets):
        super(Covid_Dataset, self)
        self.data = data
        self.targets = targets
    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]
    def __len__(self):
        return len(self.targets)


In [ ]:
# Créer les 3 objets en instantiant votre classe
train_dataset = Covid_Dataset(train_data, train_targets)
test_dataset = Covid_Dataset(test_data, test_targets)
validation_dataset = Covid_Dataset(validation_data, validation_targets)

In [ ]:
# Importer DataLoader de torch.utils.data
from torch.utils.data import DataLoader

In [ ]:
# Créer une variable pour la taille du batch
batch_size = 64

In [ ]:
# Créer les objets DataLoader pour vos datasets d'apprentissage, test et validation en lui donner la taille du batch convenue

train_DL = DataLoader(train_dataset, batch_size=batch_size)
test_DL = DataLoader(test_dataset, batch_size=batch_size)
validation_DL = DataLoader(validation_dataset, batch_size=batch_size)


In [ ]:
len(train_DL)

In [ ]:
# Importer le module nn
import torch.nn as nn

In [ ]:
train_data.shape[1]

In [ ]:
# En utilisant Sequential(), créer un modèle avec l'architecture susmentionnée
CovidNN = nn.Sequential(nn.Linear(train_data.shape[1], 64),nn.ReLU(),nn.Linear(64, 2))

In [ ]:
# Définir la fonction du coût. On peut choisir CrossEntropyLoss
loss_function = nn.CrossEntropyLoss()

In [ ]:
# Définir une fonction d'optimisation des coût: Adam par exemple. On devra définir un learning rate. On choisira 0.001.
import torch.optim as optim

optimizer = optim.Adam(CovidNN.parameters(), lr=0.0001)

In [ ]:
for data,targets in train_DL:
  print(data)
  break

In [ ]:
# Créer une boucle sur les epochs:
def training(model,train_DL,validation_DL,nb_epochs,optimizer,loss_function):
    for i in range(nb_epochs):
        # Spécifier qu'on est sur le mode entraînement
        model.train()
        # initialiser notre coût d'apprentissage à 0.0
        cout_appr = 0
        # Boucler sur les minibatchs des données d'entaînement (les données et leurs targets):
        for data,targets in train_DL:    
            # le vecteur des labels prédites par le modèle est le résultat de l'application du modèle sur le minibatch en cours. 
            output = model(data.float())
            # Calculer le coût en comparant les labels prédits aux targets du minibatch
            loss = loss_function(output,targets)
            # Backpropagation: 
            # Réinitialiser l'optimiseur
            optimizer.zero_grad()
            # Faire la backpropagation
            loss.backward()
            # Effectuer un pas d'optimisation
            optimizer.step()
            # Mettre à jour votre coût d'apprentissage en lui ajoutant le coût du data batch
            cout_appr += loss.item()
        # A la sortie de la boucle de l'entraînement, on calcule le coût moyen pour toutes les données training
        cout_moyen = cout_appr/len(train_DL.dataset)
        print('Train_Loss:',cout_moyen)
        # Initiliser le coût de validation à 0.0
        cout_valid = 0
        # Initialiser le nombre de prévisions correctes à 0
        prevision_correcte = 0
        # Spécifier qu'on est sur le mode d'évaluation
        model.eval()
        # Indiquer à Pytorch qu'on ne va pas faire de Gradient descent (comme on est dans l'évaluation)
        with torch.no_grad():
            # Boucler sur les minibatchs des données de validation (les données et leurs targets):
            for data,targets in validation_DL:  
                # le vecteur des labels prédites par le modèle est le résultat de l'application du modèle sur le minibatch en cours. 
                output = model(data.float())
                # Calculer le coût en comparant les labels prédits aux targets du minibatch
                loss = loss_function(output,targets)
                # Mettre à jour votre coût de validation en lui ajoutant le coût du data batch
                cout_valid += loss.item()
                # Mettre à jour le nombre de prévision correctes en y ajoutant le nombre des bonnes prévision sur ce batch
                # On y compare le label prédit avec le labels du minibatch. 
                # Penser à utiliser argmax pour avoir la prévision finale à partir du vecteur de prévision
                #prevision_correcte += len(output[output == targets])
                prevision_correcte += torch.sum((torch.argmax(output, dim=1) == targets)).item()
            # A la sortie de cette boucle, calculer le coût moyen de validation
            cout_valid_moyenne = cout_valid/len(validation_DL.dataset)
            print('Valid_Loss:',cout_valid_moyenne)
            # Calculer la précision: la moyenne des prévisions correctes sur l'ensemble des observations dans le dataset validation 
            prevision_correcte_moyenne = prevision_correcte/len(validation_DL.dataset)
            print('Accuracy:',prevision_correcte_moyenne*100,'%\n')
            
        # Afficher pour chaque itération le coût d'entraînement, le coût de validation, et la précision.



In [ ]:
for layer in CovidNN.children():
    if hasattr(layer, 'reset_parameters'):
        layer.reset_parameters()

In [ ]:
training(CovidNN,train_DL,validation_DL,9,optimizer,loss_function)

In [31]:
# Initiliser le coût de test à 0.0
cout_test = 0
# Initialiser le nombre de prévisions correctes à 0
prevision_correcte = 0
with torch.no_grad():
    # Boucler sur les minibatchs des données de test (les données et leurs targets):
    for data,targets in test_DL:  
        # le vecteur des labels prédites par le modèle est le résultat de l'application du modèle sur le minibatch en cours. 
        output = CovidNN(data.float())
        # Calculer le coût en comparant les labels prédits aux targets du minibatch
        loss = loss_function(output,targets)
        # Mettre à jour votre coût de test en lui ajoutant le coût du data batch
        cout_test += loss.item()
        # Mettre à jour le nombre de prévision correctes en y ajoutant le nombre des bonnes prévision sur ce batch
        # On y compare le label prédit avec le labels du minibatch. 
        # Penser à utiliser argmax pour avoir la prévision finale à partir du vecteur de prévision
        #prevision_correcte += len(output[output == targets])
        prevision_correcte += torch.sum((torch.argmax(output, dim=1) == targets)).item()
    # A la sortie de cette boucle, calculer le coût moyen de test
    cout_test_moyenne = cout_test/len(test_dataset)

    # Calculer la précision: la moyenne des prévisions correctes sur l'ensemble des observations dans le dataset test 
    prevision_correcte_moyenne = prevision_correcte/len(test_dataset)

In [32]:
prevision_correcte_moyenne

0.9088785046728972

In [33]:
len(test_dataset)

1284

In [34]:
len(test_DL.dataset)

1284

In [35]:
# sauvegarder le modèle dans un fichier pickle
#torch.save(CovidNN,'CovidNN.pkl')
#torch.save(CovidNN,'interface_graphique/CovidNN.pkl')

In [36]:
def testModel(model,test_DL,loss_function):
    # Initiliser le coût de test à 0.0
    cout_test = 0
    # Initialiser le nombre de prévisions correctes à 0
    prevision_correcte = 0
    with torch.no_grad():
        # Boucler sur les minibatchs des données de test (les données et leurs targets):
        for data,targets in test_DL:  
            # le vecteur des labels prédites par le modèle est le résultat de l'application du modèle sur le minibatch en cours. 
            output = model(data.float())
            # Calculer le coût en comparant les labels prédits aux targets du minibatch
            loss = loss_function(output,targets)
            # Mettre à jour votre coût de test en lui ajoutant le coût du data batch
            cout_test += loss.item()
            # Mettre à jour le nombre de prévision correctes en y ajoutant le nombre des bonnes prévision sur ce batch
            # On y compare le label prédit avec le labels du minibatch. 
            # Penser à utiliser argmax pour avoir la prévision finale à partir du vecteur de prévision
            #prevision_correcte += len(output[output == targets])
            prevision_correcte += torch.sum((torch.argmax(output, dim=1) == targets)).item()
        # A la sortie de cette boucle, calculer le coût moyen de test
        cout_test_moyenne = cout_test/len(test_DL.dataset)

        # Calculer la précision: la moyenne des prévisions correctes sur l'ensemble des observations dans le dataset test 
        prevision_correcte_moyenne = prevision_correcte/len(test_DL.dataset)
        print("Testing results :\nLoss : ",cout_test_moyenne,"\nAccuracy : ",prevision_correcte_moyenne*100,"%\n")

In [37]:
def createModel(dataDF,targets,batch_size,neuronesNb,learning_rate,epochs,seuil1=0,seuil2=0):
    
    newDF = redDim(dataDF,seuil1,seuil2)
    data=torch.tensor(newDF.to_numpy())


    # Créer 4 tensors en résultat du splitting: données d'apprentissage, données de test, labels d'apprentissage, et labels de test  
    atrain_data, test_data, atrain_targets, test_targets = train_test_split(data, targets, test_size=0.2)

    # Créer 4 tensors en résultat du splitting: données d'apprentissage, données de validation, labels d'apprentissage, et labels de validation  
    train_data, validation_data, train_targets, validation_targets = train_test_split(atrain_data, atrain_targets, test_size=1000)

    # Créer les 3 objets en instantiant votre classe
    train_dataset = Covid_Dataset(train_data, train_targets)
    test_dataset = Covid_Dataset(test_data, test_targets)
    validation_dataset = Covid_Dataset(validation_data, validation_targets)


    # Créer les objets DataLoader pour vos datasets d'apprentissage, test et validation en lui donner la taille du batch convenue

    train_DL = DataLoader(train_dataset, batch_size=batch_size)
    test_DL = DataLoader(test_dataset, batch_size=batch_size)
    validation_DL = DataLoader(validation_dataset, batch_size=batch_size)

    # En utilisant Sequential(), créer un modèle avec l'architecture susmentionnée
    CovidNN = nn.Sequential(nn.Linear(train_data.shape[1], neuronesNb),nn.ReLU(),nn.Linear(neuronesNb, 2))

    # Définir la fonction du coût. On peut choisir CrossEntropyLoss
    loss_function = nn.CrossEntropyLoss()

    optimizer = optim.Adam(CovidNN.parameters(), lr=learning_rate)
    

    for layer in CovidNN.children():
        if hasattr(layer, 'reset_parameters'):
            layer.reset_parameters()

    training(CovidNN,train_DL,validation_DL,epochs,optimizer,loss_function)

    testModel(CovidNN,test_DL,loss_function)

model = createModel(newDF,0,9000,64,9,0.001,64)

In [40]:
#model = createModel(newDF,targets,32,64,0.0001,8)

In [ ]:
# sauvegarder le modèle dans un fichier pickle
#modelFileName='CovidNN.tkl'
#torch.save(CovidNN,modelFileName)
#torch.save(CovidNN,'interface_graphique/'+modelFileName)